# Dimensionality reduction

## Data

Donwload and parse data:

In [1]:
!cd data && ./download.sh

--2017-09-28 02:15:20--  http://mit.spbau.ru/sewiki/images/8/87/Wikipedia_2000_dump.xml.gz
Resolving mit.spbau.ru... 194.85.238.20
Connecting to mit.spbau.ru|194.85.238.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13902445 (13M) [application/x-gzip]
Saving to: ‘Wikipedia_2000_dump.xml.gz’

Wikipedia_2000_dump 100%[===================>]  13.26M  4.28MB/s    in 3.1s    

2017-09-28 02:15:23 (4.28 MB/s) - ‘Wikipedia_2000_dump.xml.gz’ saved [13902445/13902445]



In [2]:
DATA_FILE="data/Wikipedia_2000_dump.xml"

In [3]:
import xml.etree.ElementTree as ET
import pandas as pd


def xml2df(xml_data):
    root = ET.XML(xml_data)
    all_records = []
    for child in root:
        record = {}
        for name, value in child.attrib.items():
            record[name] = value
        record["content"] = child.text
        all_records.append(record)
    return pd.DataFrame(all_records)

In [4]:
data = xml2df(open(DATA_FILE).read())["content"].tolist()

In [5]:
data[0][:500]

'Литва́ , официальное название\xa0— Лито́вская Респу́блика \xa0— государство, географически расположенное в Северной Европе (Прибалтика). Столица страны\xa0— Вильнюс.\n\nРасположена на восточном побережье Балтийского моря. На севере граничит с Латвией, на юго-востоке\xa0— с Белоруссией, на юго-западе\xa0— c Польшей и Калининградской областью России.\n\nЧлен ООН с 1991 года, ЕС и НАТО\xa0— с 2004 года. Входит в Шенгенскую зону и Еврозону.\n\nПоверхность\xa0— равнинная со следами древнего оледенения. Поля и луга занимают 57\xa0'

## Vectorization

Preprocess data, then vectorize it using simple BOW model:

In [6]:
import string
from nltk import sent_tokenize, wordpunct_tokenize
from sklearn.base import BaseEstimator, TransformerMixin


class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._punct = set(string.punctuation + "«»№")
    
    def fit(self, X, y=None):
        return self
    
    def _filter_gen(self, text):
        text = "".join(filter(lambda c: c != '́', text))
        for sent in sent_tokenize(text):
            for word in wordpunct_tokenize(sent):
                if word.isalpha():
                    yield word.lower()
    
    def _tokenize(self, text):
        return list(self._filter_gen(text))
            
    def transform(self, X):
        return list(" ".join(self._tokenize(text)) for text in X)

In [7]:
preprocessor = Preprocessor()
data[0][:500]

'Литва́ , официальное название\xa0— Лито́вская Респу́блика \xa0— государство, географически расположенное в Северной Европе (Прибалтика). Столица страны\xa0— Вильнюс.\n\nРасположена на восточном побережье Балтийского моря. На севере граничит с Латвией, на юго-востоке\xa0— с Белоруссией, на юго-западе\xa0— c Польшей и Калининградской областью России.\n\nЧлен ООН с 1991 года, ЕС и НАТО\xa0— с 2004 года. Входит в Шенгенскую зону и Еврозону.\n\nПоверхность\xa0— равнинная со следами древнего оледенения. Поля и луга занимают 57\xa0'

In [8]:
preprocessor.transform([data[0][:500]])[0]

'литва официальное название литовская республика государство географически расположенное в северной европе прибалтика столица страны вильнюс расположена на восточном побережье балтийского моря на севере граничит с латвией на юго востоке с белоруссией на юго западе c польшей и калининградской областью россии член оон с года ес и нато с года входит в шенгенскую зону и еврозону поверхность равнинная со следами древнего оледенения поля и луга занимают'

In [9]:
from sklearn.feature_extraction.text import CountVectorizer as BagOfWords
from sklearn.pipeline import make_pipeline


model = make_pipeline(
    Preprocessor(),
    BagOfWords()
)

In [10]:
X = model.fit_transform(data)
X.shape

(2000, 283507)

In [11]:
X

<2000x283507 sparse matrix of type '<class 'numpy.int64'>'
	with 1788050 stored elements in Compressed Sparse Row format>

## Reduction

Calculate erank, then do the reduction using LSA:

In [12]:
import math
from scipy.sparse import linalg
from scipy import stats


def erank(M):
    u = linalg.svds(M.astype(float), k=min(M.shape) - 1, return_singular_vectors=False)
    return math.exp(stats.entropy(u / sum(u)))

In [13]:
e = erank(X)
m = int(round(e))
m

1049

In [14]:
from sklearn.decomposition import TruncatedSVD



X_reduced = TruncatedSVD(n_components=m, algorithm="arpack").fit_transform(X.astype(float))
X_reduced.shape

(2000, 1049)